In [1]:
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


c:\Users\pacie\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Carregar o CSV 
import os


file_path = os.path.join('..', 'data_config', 'modified_hotel_reservations.csv')
df = pd.read_csv(file_path)
# Testando sem as colunas id e status
df = df.drop(columns=['Booking_ID', 'booking_status'])
df['no_total_people'] = df['no_of_adults'] + df['no_of_children']
df['no_total_nights'] = df['no_of_weekend_nights'] + df['no_of_week_nights']


In [3]:
# Separar as features e o label
X = df.drop('label_avg_price_per_room', axis=1)
y = df['label_avg_price_per_room']

# Identificar colunas categóricas e numéricas
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['number']).columns

# Criar os transformers para os pipelines de pré-processamento
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')


In [4]:
# Criar o pré-processador com ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Definir a função objetivo para o estudo do Optuna
from sklearn.ensemble import RandomForestClassifier

# Criar o modelo RandomForestClassifier com os hiperparâmetros aplicáveis
rf = RandomForestClassifier(
    n_estimators=500,
    random_state=42
)

# Criar o pipeline incluindo o pré-processador e o modelo com RandomForestClassifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', rf)
])


In [5]:
# Treinando o modelo com os melhores hiperparâmetros encontrados pelo GridSearchCV para XGBClassifier
model.fit(X_train, y_train)

# Prever nos dados de teste e calcular a acurácia com o modelo treinado
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8585290550226045
Classification Report:
               precision    recall  f1-score   support

           1       0.88      0.86      0.87      2757
           2       0.81      0.84      0.83      3367
           3       0.89      0.88      0.89      2945

    accuracy                           0.86      9069
   macro avg       0.86      0.86      0.86      9069
weighted avg       0.86      0.86      0.86      9069

